In [1]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('/content/books.csv',error_bad_lines=False)

<ipython-input-2-a8dcd3b67361>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('/content/books.csv',error_bad_lines=False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



In [3]:
books

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [4]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


In [5]:
#converting text to numerical vectors in book titles
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_new = vectorizer.fit_transform([x.lower() for x in books['title']])

In [6]:
#converting the query into numerical vector
query = 'orange'
query_vec = vectorizer.transform([query])

In [7]:
#calculating the cosine similarity between the query and book titles
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(query_vec, X_new).flatten()

In [8]:
#sorting the results
test = np.argsort(-similarity)
result = books.iloc[test]
result['title'].head()

6770        The Orange Girl
10946           Code Orange
2416     A Clockwork Orange
2417     A Clockwork Orange
2418       Clockwork Orange
Name: title, dtype: object

In [9]:
len(result)

11123

As we can see above, the len of results is similar to the number of records in the original dataframe, which means our search engine is showing all the records which is not very practical to users. In order to limit the search result, first we can remove the results that has low cosine similarity.

Then we can do feature engineering. For example two features that we own are avg_rating and rating_count. We can multiply these two numbers and assign a new score to the results and sort results based on that.

In [10]:
#remove the unrelevent results
match_idx = np.where(similarity != 0)[0]
indices = np.argsort(-similarity[match_idx])
correct_indices = match_idx[indices]
result = books.iloc[correct_indices]
result['title'].head()

6770        The Orange Girl
10946           Code Orange
2416     A Clockwork Orange
2417     A Clockwork Orange
2418       Clockwork Orange
Name: title, dtype: object

In [11]:
result['title']

6770                                       The Orange Girl
10946                                          Code Orange
2416                                    A Clockwork Orange
2417                                    A Clockwork Orange
2418                                      Clockwork Orange
10716                                     Tropic of Orange
4219                           Five Quarters of the Orange
4220                           Five Quarters of the Orange
5568                               The Burnt Orange Heresy
2419                       A Clockwork Orange (Stage Play)
117      The Burnt Orange Heresy (Vintage Crime/Black L...
Name: title, dtype: object

In [12]:
len(result)

11

In [13]:
books['score'] = np.log(books['average_rating'] *  books['ratings_count'])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
result = books.iloc[correct_indices]
overall = result['score']  * similarity[correct_indices]
result = result.loc[overall.sort_values(ascending=False).index]

In [18]:
result['title']

6770                                       The Orange Girl
10946                                          Code Orange
2416                                    A Clockwork Orange
4219                           Five Quarters of the Orange
2417                                    A Clockwork Orange
10716                                     Tropic of Orange
4220                           Five Quarters of the Orange
5568                               The Burnt Orange Heresy
2418                                      Clockwork Orange
2419                       A Clockwork Orange (Stage Play)
117      The Burnt Orange Heresy (Vintage Crime/Black L...
Name: title, dtype: object

In order to have it all together, we can write a function that:

1) convert the query (which is alphabetical and lowercase) into vector

2) calculate the cosine similarity between the query and other titles

3) Remove the titles that has 0 similarity

4) Multiply titles's scores (avgRating*numberOfRatings) into their similarity

5) Sort the result

In [25]:
import re

def search_engine(word, limit=15):
    word = re.sub('[^a-zA-Z0-9 ]','', word.lower()) # match everyting that's not alphabet and digit and remove it
    query_vec = vectorizer.transform([word])
    similarity = cosine_similarity(query_vec, X_new).flatten()

    filtered = np.where(similarity != 0)[0]
    indices = np.argsort(-similarity[filtered])
    correct_indices = filtered[indices]
    result = books.iloc[correct_indices]

    if not len(result):
        return 'result not found'

    overall =  result['score'] *  similarity[correct_indices]

    return result['title'].loc[overall.sort_values(ascending=False).index].head(limit)

In [32]:
user_input = input("Search for books titles: ")
output = search_engine(user_input)
print(output)

Search for books titles: Orange
6770                                       The Orange Girl
10946                                          Code Orange
2416                                    A Clockwork Orange
4219                           Five Quarters of the Orange
2417                                    A Clockwork Orange
10716                                     Tropic of Orange
4220                           Five Quarters of the Orange
5568                               The Burnt Orange Heresy
2418                                      Clockwork Orange
2419                       A Clockwork Orange (Stage Play)
117      The Burnt Orange Heresy (Vintage Crime/Black L...
Name: title, dtype: object
